In [3]:
# Necessary imports
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, IntRangeSlider, interactive
from IPython.display import display, clear_output
from commpy.channels import awgn
import time

# Parameters
bits_per_symbol = 4  # bits per symbol
num_symbols = 1000  # number of simulated symbols
num_samples = 80  # number of samples per symbol (oversampling)

# Derived parameters
M = 2 ** bits_per_symbol  # number of different symbols
baud_rate = 1  # Baud Rate
carrier_freq = 2 * M * baud_rate  # RF frequency

# Derived parameters
num_bits = bits_per_symbol * num_symbols  # number of simulated data bits
symbol_period = 1 / baud_rate  # one symbol period
sampling_period = symbol_period / num_samples  # oversampling period

# M frequencies in "coherent" distance (BR)
frequencies = carrier_freq + (baud_rate / 2) * (np.arange(1, M + 1) - (M + 1) / 2)

# Calculate the maximum frequency
max_frequency = np.max(frequencies)

# Recalculate num_samples to ensure Fs = num_samples * BR > 2 * max_frequency
sampling_freq = 2 * max_frequency
num_samples = int(np.ceil(sampling_freq / baud_rate)) + 10

# Recalculate the oversampling period
sampling_period = symbol_period / num_samples

# Input data bits
bits = np.random.randint(0, 2, num_bits)
bit_matrix = bits.reshape((num_symbols, bits_per_symbol))

time_vector = np.arange(0, len(bit_matrix) * symbol_period, symbol_period)  # time vector on the T grid
oversampling_time_vector = np.arange(0, symbol_period, sampling_period)  # oversampling time vector

# FSK signal
fsk_signal = []
amplitude = np.sqrt(2 / symbol_period / num_samples)
for symbol_index in range(len(bit_matrix)):
    freq_symbol = frequencies[int(''.join(map(str, bit_matrix[symbol_index])), 2)]
    time_segment = (symbol_index * symbol_period) + oversampling_time_vector
    fsk_signal.append(np.sin(2 * np.pi * freq_symbol * time_segment))
fsk_signal = np.concatenate(fsk_signal)


def calculate_errors(EbNo_range):
    clear_output(wait=True)
    EbNo_values = list(range(EbNo_range[0], EbNo_range[1] + 1))
    errors_list = []
    for EbNo in EbNo_values:
        # Calculate SNR
        SNR = EbNo + 10 * np.log10(bits_per_symbol) - 10 * np.log10(num_samples / 2)  # in dB

        # Add noise to the FSK (passband) signal
        noisy_signal = awgn(fsk_signal, SNR)

        # FSK receiver
        received_symbols = []
        for symbol_index in range(len(noisy_signal) // num_samples):
            time_segment = (symbol_index * symbol_period) + oversampling_time_vector
            signal_segment = noisy_signal[symbol_index * num_samples:(symbol_index + 1) * num_samples]
            match_scores = []
            for freq in frequencies:
                signal_template = np.sin(2 * np.pi * freq * time_segment)
                match_scores.append(np.sum(signal_segment * signal_template))
            decoded_symbol = np.argmax(match_scores)
            received_symbols.append([int(bit) for bit in bin(decoded_symbol)[2:].zfill(bits_per_symbol)])
        received_symbols = np.array(received_symbols).reshape((num_symbols, bits_per_symbol))

        # Count errors
        errors = np.sum(bit_matrix != received_symbols)
        errors_list.append(errors)

    # Plot the results
    plt.figure(figsize=(10, 6))
    plt.plot(EbNo_values, errors_list, marker='o', linestyle='-', markersize=8)
    plt.xlabel('Eb/No (dB)')
    plt.ylabel('Number of Errors')
    plt.title('Number of Errors vs. Eb/No')
    plt.grid(True)
    plt.show()

# Create the range slider widget
EbNo_slider = IntRangeSlider(
    value=[0, 20],
    min=0,
    max=20,
    step=1,
    description='EbNo (dB):',
    continuous_update=False,
    layout=Layout(width='99%')
)

# Create interactive widget
interactive_plot = interactive(calculate_errors, EbNo_range=EbNo_slider)

input_widgets = VBox([EbNo_slider], layout=Layout(flex='1 1 auto', width='auto'))
plot_output = interactive_plot.children[-1]  # The output plot

# Create a VBox that includes both the input widgets and the plot
ui = VBox([input_widgets, plot_output])

# Display the UI components
clear_output(wait=True)  # Clear the previous output
display(ui)
